In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd
import numpy as np
import os

sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

# auto-width
from auto_width8 import auto_columns_width


from universal_connection import UniversalConnection, DBType
from report_utils import query_by_name
from log import log

In [2]:
from region import get_region_by_two_bases, clean_region

In [3]:
conn = UniversalConnection('../../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df: pd.DataFrame = query_by_name(conn, os.getcwd(), 'sql_all')
conn.close()

2020-07-17 18:29:06 connect to postgres database using config file "../../../../.credentials/telemed/prom.cfg"
2020-07-17 18:29:06 creating ssh tunnel to 172.16.100.19 as root...
2020-07-17 18:29:06 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 41943
2020-07-17 18:29:06 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-07-17 18:29:06 @telemed query:
                    with temp as (
				    select	product.id,
				    		product.partner_id,
				    		product.full_name,
				    		pc.id product_condition_id,
				    		promo.id promo_id,
				    		pp.patient_id patient_id--,
				    		--a.id app_id,
				    		--a.price ,
				    		--product.full_name
				    		--concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				            --p_patient.birthday patient_birthday,
				            --p_patient.phone,
	

In [4]:
df.iloc[0]

patient_id           NaN
patient_fio             
patient_birthday    None
phone               None
email               None
sex                 None
app_id               NaN
full_name           None
cons_date           None
specialty           None
Name: 0, dtype: object

In [5]:
df = df[1:].reset_index(drop=True)
df['app_id'].fillna(0, inplace=True)
df['phone'].fillna('', inplace=True)
df['email'].fillna('', inplace=True)
df['sex'].fillna('', inplace=True)
df['full_name'].fillna('', inplace=True)


df.patient_id = df.patient_id.astype(int)
df.app_id = df.app_id.astype(int)

df['dirty_region'] = df['phone'].apply(get_region_by_two_bases)
df['region'] = df['dirty_region'].apply(clean_region)

In [6]:
df

,patient_id,patient_fio,patient_birthday,phone,email,sex,app_id,full_name,cons_date,specialty,dirty_region,region
0,3501140,Буланкина Екатерина Михайловна,1991-05-27,79638058150,,FEMALE,151805,Телемедицина_ВЭБ,2020-04-20,Акушер-гинеколог,Амурская обл.,Амурская область
1,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,275480,Телемедицина_ВЭБ,2020-06-20,Терапевт,Иркутская обл.,Иркутская область
2,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,275489,Телемедицина_ВЭБ,2020-06-22,Дерматолог,Иркутская обл.,Иркутская область
3,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,278161,Телемедицина_ВЭБ,2020-06-23,Невролог,Иркутская обл.,Иркутская область
4,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,282083,Телемедицина_ВЭБ,2020-06-24,Кардиолог,Иркутская обл.,Иркутская область
...,...,...,...,...,...,...,...,...,...,...,...,...
144471,3752852,Федорова Марина,1983-12-21,79533680975,,FEMALE,0,,None,None,г. Санкт-Петербург и Ленинградская область,Санкт-Петербург
144472,3512588,Вязьмина Мария Алексеевна,2004-08-17,79086448240,,FEMALE,157545,Телемедицина_ВЭБ,2020-04-23,Акушер-гинеколог,Иркутская обл.,Иркутская область
144473,4630605,Бескоровайная Алена Сергеевна,1982-09-19,79095005003,,FEMALE,0,,None,None,Алтайский край,Алтайский край
144474,3558423,Мамаева Зарема Магомедовна,1950-10-31,79854422057,zarema.mamaeva@mail.ru,FEMALE,0,,None,None,г. Москва и Московская область,Москва и Московская область


In [7]:
len(df.patient_id.unique())

77904

In [8]:
test = df[df['app_id'] > 0]#
test.groupby(['app_id'])[['patient_id']].count()#.reset_index().rename(columns={'patient_id':'quantity'})

,patient_id
app_id,
116252,1
118293,1
118296,1
120237,1
123496,1
...,...
301729,1
301746,1
301750,1


In [9]:
test = df[df['app_id'] > 0].groupby(['app_id'])[['patient_id']].count().reset_index().rename(columns={'patient_id':'quantity'})

In [10]:
test[test['quantity'] > 1]

,app_id,quantity


In [11]:
df.specialty = df.specialty.str.replace('\<', '').str.replace('\>', '').str.replace('.', '')
df.specialty = df.specialty.str.replace('-ТМ', '')
df.specialty = df.specialty.fillna('Unknown')

In [12]:
#df

In [13]:
spec_list = list(df.specialty.unique())
spec_list.sort()
spec_list.insert(0, 'Всего консультаций')
spec_list

['Всего консультаций',
 'Unknown',
 'Акушер-гинеколог',
 'Аллерголог-иммунолог',
 'Врач УЗИ',
 'Врач общей практики (Семейный врач)',
 'Гастроэнтеролог',
 'Гематолог',
 'Дерматолог',
 'Детский кардиолог',
 'Диетолог',
 'Инструктор по ЗОЖ и фитнесу',
 'Кардиолог',
 'Консультант по питанию',
 'Косметолог',
 'Невролог',
 'Онколог',
 'Онкология',
 'Оториноларинголог',
 'Офтальмолог',
 'Педиатр',
 'Психолог',
 'Психотерапевт',
 'Пульмонолог',
 'Ревматолог',
 'Ревматология',
 'Специалист по диагностике',
 'Стоматолог терапевт',
 'Терапевт',
 'Травматолог-ортопед',
 'УЗИ',
 'Уролог',
 'Хирург',
 'Эндокринолог']

In [14]:
len(df.patient_id.unique())

77904

In [15]:
d_spec = dict()

for pat_id in list(df.patient_id.unique()):
    
    temp = df[df['patient_id'] == pat_id].reset_index(drop=True)
    #print(d_spec)
    
    d_spec[temp['patient_id'][0]] = d_spec.get(temp['patient_id'][0], [0 for specialization in spec_list])
    
    if temp['app_id'][0] != 0:
        d_spec[temp['patient_id'][0]][0] += 1
        d_spec[temp['patient_id'][0]][spec_list.index(temp['specialty'][0])] += 1 

    if temp.shape[0] > 1:
        
        for i in range(1, temp.shape[0]):
            
            if temp['app_id'][i] != 0:
                
                d_spec[temp['patient_id'][0]][0] += 1
                d_spec[temp['patient_id'][i]][spec_list.index(temp['specialty'][i])] += 1  

            
spec_report = pd.DataFrame.from_dict(d_spec, orient='index').sort_index()
spec_report.columns = spec_list
spec_report = spec_report.reset_index().rename(columns={'index':'patient_id'})
spec_report = spec_report.reset_index(drop=True)
spec_report.drop('Unknown', axis=1, inplace=True)
            
#spec_report.reset_index().head()

In [16]:
spec_report

,patient_id,Всего консультаций,Акушер-гинеколог,Аллерголог-иммунолог,Врач УЗИ,Врач общей практики (Семейный врач),Гастроэнтеролог,Гематолог,Дерматолог,Детский кардиолог,...,Ревматолог,Ревматология,Специалист по диагностике,Стоматолог терапевт,Терапевт,Травматолог-ортопед,УЗИ,Уролог,Хирург,Эндокринолог
0,611,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,973,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,979,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,981,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77899,4935753,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
77900,4935920,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77901,4935945,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77902,4936116,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df.groupby(['patient_id', 'patient_fio', 'patient_birthday', 'phone', 'email', 'sex', 'region'])[['app_id']].count().reset_index().drop('app_id', axis=1)

,patient_id,patient_fio,patient_birthday,phone,email,sex,region
0,611,Лебедева Ольга Сергеевна,2017-08-21,79067166882,oslebedeva@mail.ru,FEMALE,Москва и Московская область
1,973,Арутюнян Айрапет Люксимбургович,2000-01-01,79300889210,tucoom@mail.ru,MALE,Белгородская область
2,977,Рыбалко Анна Александровна,1989-12-30,79207884295,arubalko@yandex.ru,FEMALE,Тульская область
3,979,Жидкова Людмила Андреевна,1989-04-04,79282144524,lyudmila.jidkova2015@yandex.ru,FEMALE,Ростовская область
4,981,Магомедова Марьян Магомедовна,1982-02-19,79034777877,marian0909098476@mail.ru,FEMALE,Республика Дагестан
...,...,...,...,...,...,...,...
77899,4935753,Хакимова Светлана Владимировна,1987-10-12,79067734978,,FEMALE,Москва и Московская область
77900,4935920,Бечейкин Илья Сергеевич,1989-12-02,79037707962,mgupiy@yandex.ru,MALE,Москва и Московская область
77901,4935945,Июнина Дарья Евгеньевна,1994-05-31,79397011551,,FEMALE,Самарская область
77902,4936116,Козева Юлия,1998-03-31,79081785071,,FEMALE,Ростовская область


In [18]:
df_pat = df.groupby(['patient_id', 'patient_fio', 'patient_birthday', 'phone', 'email', 'sex', 'region'])[['app_id']].count().reset_index().drop('app_id', axis=1)

In [19]:
df_total = df_pat.merge(spec_report)

In [20]:
global writer
writer = None

writer = auto_columns_width(os.getcwd() + '/data_all' + '.xlsx',
                            df.sort_values(by='patient_id'), 
                            'Данные',
                            writer,
                            1.2)

writer = auto_columns_width(os.getcwd() + '/data_all' + '.xlsx',
                            df_total, 
                            'Данные по врачам',
                            writer,
                            1.2)

writer.save()